In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime
import random
import crossval #this is the file Eli wrote



from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier, RandomForestClassifier, BaggingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, balanced_accuracy_score, plot_precision_recall_curve, precision_recall_curve, precision_score, recall_score

from sklearn.model_selection import RandomizedSearchCV, train_test_split
import shap
import lightgbm as lgb 

pd.set_option('max_colwidth', 80)

In [4]:
df = pd.read_csv('final_data.csv') 
df.head()

,TARGET_FID,Length,Install_year,Arterial_class,Surface_width,Speed_limit,Slope,Width,Break_Yr,Target,...,Surface_type_,Surface_type_AC,Surface_type_AC/AC,Surface_type_AC/PCC,Surface_type_GRAVEL,Surface_type_PCC,Surface_type_ST,AGE,Nearby_breaks_1yr,Process_year
0,1,314.79,1972,2.0,40.0,25.0,0.0,8.0,NaN,0,...,0,0,0,0,0,1,0,38,0,2010
1,2,363.39,1972,0.0,46.0,20.0,4.0,8.0,NaN,0,...,0,1,0,0,0,0,0,38,0,2010
2,3,323.51,1972,0.0,46.0,20.0,4.0,8.0,NaN,0,...,0,1,0,0,0,0,0,38,0,2010
3,4,329.13,1928,0.0,0.0,20.0,6.0,12.0,NaN,0,...,0,0,0,0,0,0,1,82,0,2010
4,5,273.64,1928,2.0,42.0,25.0,4.0,18.0,NaN,0,...,0,0,0,1,0,0,0,82,0,2010


In [8]:
param_list = {
      'n_estimators': [50],
      'max_depth': [30]
    } 
param = {}
for key in param_list.keys():
    val = random.choice(param_list[key])
    param[key] = val

In [9]:
# Used to build a list containing dictionaries w every compination of params
# from param list, start with a list containing one dict (src), 
# then call for each param (cross, keyword)
def cross_params(src, cross, keyword):
    combos = []
    for d in src:
        for val in cross:
            cpy = d.copy()
            cpy[keyword] = val
            combos.append(cpy)
    return combos

# Uses cross_params to get list of all parameter combinations 
def get_param_combos(param_list):
    param_combos = [{}]
    for kw in param_list:
        param_combos = cross_params(param_combos, param_list[kw], kw)
    return param_combos


In [10]:
def rand_param_search(data, param_list, classifier_fn, n_val=2, threshold=0.5, print_results=True):
    """

    """
    best_score = -1.0 
    best_param = {}
    param_combos = get_param_combos(param_list)
    for param in param_combos:
        print("using the following parameters:")
        print(param)
        avgs = crossval.crossval_by_year(data, classifier_fn, param, n_val, threshold, print_results)
        print(avgs[0])
        print(avgs[1])
        print(avgs[2])
        print()
        if avgs[0] > best_score:
            best_param = param
            best_score = avgs[0]
    
    
    return best_param, best_score

In [12]:
for i in range(1, 11):
    t = 0.35 + i /100    
    best_score, best_param = rand_param_search(df, param_list, RandomForestClassifier, threshold=t)
    print(t)
    print(best_param)
    print(best_score)

using the following parameters:
{'n_estimators': 50, 'max_depth': 30}
Cross-validation iteration 1 of 11 results:
Balanced accuracy = 0.6789
Recall            = 0.5245
Precision         = 0.8333

Cross-validation iteration 2 of 11 results:
Balanced accuracy = 0.6473
Recall            = 0.4066
Precision         = 0.8880

Cross-validation iteration 3 of 11 results:
Balanced accuracy = 0.6632
Recall            = 0.3814
Precision         = 0.9451

Cross-validation iteration 4 of 11 results:
Balanced accuracy = 0.6631
Recall            = 0.3939
Precision         = 0.9324

Cross-validation iteration 5 of 11 results:
Balanced accuracy = 0.6757
Recall            = 0.4028
Precision         = 0.9486

Cross-validation iteration 6 of 11 results:
Balanced accuracy = 0.6680
Recall            = 0.4065
Precision         = 0.9295

Cross-validation iteration 7 of 11 results:
Balanced accuracy = 0.6677
Recall            = 0.3770
Precision         = 0.9583

Cross-validation iteration 8 of 11 results:
Bala